In [5]:
!pip install surprise

  Using cached surprise-0.1-py2.py3-none-any.whl (1.8 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 772.0/772.0 kB 969.1 kB/s eta 0:00:001m971.8 kB/s eta 0:00:01
  Preparing metadata (setup.py) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.3-cp310-cp310-macosx_12_0_arm64.whl size=1125168 sha256=0556a3c1278d6e647f20146a6852298d758af2d05fbe0615a0001d4c733cd146
  Stored in directory: /Users/noteorbit/Library/Caches/pip/wheels/df/e4/a6/7ad72453dd693f420b0c639bedeec34641738d11b55d8d9b84
Successfully built scikit-surprise

[notice] A new release of pip available: 22.3.1 -> 23.0.1
[notice] To update, run: pip install --upgrade pip


In [6]:
pip install --upgrade pip

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 2.2 MB/s eta 0:00:00 MB/s eta 0:00:01:010m
  Attempting uninstall: pip
    Found existing installation: pip 22.3.1
    Uninstalling pip-22.3.1:
      Successfully uninstalled pip-22.3.1
Note: you may need to restart the kernel to use updated packages.


In [123]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from surprise import accuracy, Dataset, SVD,Reader, NormalPredictor
from surprise.model_selection import train_test_split, cross_validate , GridSearchCV , KFold


In [124]:
reader = Reader(rating_scale=(1, 5))

In [125]:
data = pd.read_csv('data/real.csv',index_col=0)

In [126]:
data = data.drop_duplicates(['Store','User'])
data.head()

,Store,User,Rating,Review,Duration
0,NAP's Coffee & Roasters,Nuunet D.,5,ร้านนี้ยกให้เป็นที่หนึ่งในอุบลเลย กาแฟดีมากๆ ร...,เดือนที่แล้ว
1,NAP's Coffee & Roasters,vanai sun,3,เป็นอีกหนึ่งร้านที่ตั้งใจจะไป วันนี้ได้มาลองชิ...,10 เดือนที่แล้ว
2,NAP's Coffee & Roasters,Phonanan Seeon,5,10 10 10 บรรยากาศคือดี ใดๆคือพนักงานดีมาก เคร...,9 เดือนที่แล้ว
3,NAP's Coffee & Roasters,Chavakorn Laophetsakunchai,5,ดีเลยยยย กาแฟ หอมอร่อยยย เมล็ดก้ดีๆๆ,เดือนที่แล้ว
4,NAP's Coffee & Roasters,community man,5,กาแฟรสชาดดีมาก แต่วันนี้มีน้องๆคุยกันเสียงดังม...,ปีที่แล้ว


In [127]:
dataset = Dataset.load_from_df(data[['Store','User','Rating']],reader)

In [128]:
trainset, testset = train_test_split(dataset, test_size=0.25)


In [129]:

threshold = int(.9 * len(data))                                     
# trainset_raw_ratings = raw_ratings[:threshold]                             
# test_raw_ratings = raw_ratings[threshold:]                                 

In [130]:
# define a cross-validation iterator
kf = KFold(n_splits=5)

algo = SVD()

for trainset, testset in kf.split(dataset):

    # train and test algorithm.
    algo.fit(trainset)
    predictions = algo.test(testset)

    # Compute and print Root Mean Squared Error
    accuracy.rmse(predictions, verbose=True)

RMSE: 0.7548
RMSE: 0.7538
RMSE: 0.7750
RMSE: 0.7499
RMSE: 0.7538


In [131]:
trainset1 = data.iloc[:threshold]
testset1 = data.iloc[threshold:]

In [134]:
cross_validate(NormalPredictor(), dataset, cv=2)


{'test_rmse': array([1.02611417, 0.99686365]),
 'test_mae': array([0.76029387, 0.72064755]),
 'fit_time': (0.0019037723541259766, 0.0008618831634521484),
 'test_time': (0.011025190353393555, 0.002830028533935547)}

In [139]:
svd = SVD(n_epochs=10,n_factors=15)
cross_validate(svd, dataset, measures=['RMSE', 'MAE'], cv=3, verbose=True)

Evaluating RMSE, MAE of algorithm SVD on 3 split(s).

                  Fold 1  Fold 2  Fold 3  Mean    Std     
RMSE (testset)    0.7899  0.7611  0.7545  0.7685  0.0154  
MAE (testset)     0.6161  0.6066  0.6041  0.6090  0.0052  
Fit time          0.01    0.00    0.00    0.01    0.00    
Test time         0.01    0.00    0.00    0.01    0.00    


{'test_rmse': array([0.78994485, 0.76105184, 0.75453744]),
 'test_mae': array([0.61614913, 0.60661732, 0.6040902 ]),
 'fit_time': (0.01025390625, 0.004521846771240234, 0.004480838775634766),
 'test_time': (0.006806850433349609,
  0.004692792892456055,
  0.004812002182006836)}

In [107]:
model = SVD()

In [116]:
""" 

n_epochs is number of train
lr_all is learning rate
reg_all is regularization term for all parameters


"""

' \n\nn_epochs is number of train\nlr_all is learning rate\nreg_all is regularization term for all parameters\n\n\n'

In [120]:
param_grid = {"n_epochs": [5, 10], "lr_all": [0.002, 0.005, 0.001, 0.01], "reg_all": [0.4, 0.6, 0.7 ,0.2],"n_factors":[5,10,15,20,30]}
gs = GridSearchCV(SVD, param_grid, measures=["rmse", "mae"], cv=5)

In [121]:
gs.fit(dataset)
# best RMSE score

print(gs.best_score["rmse"])

# combination of parameters that gave the best RMSE score
print(gs.best_params["rmse"])

0.7510739792757256
{'n_epochs': 10, 'lr_all': 0.01, 'reg_all': 0.2, 'n_factors': 5}


In [122]:
results_df = pd.DataFrame.from_dict(gs.cv_results)
results_df.head()

,split0_test_rmse,split1_test_rmse,split2_test_rmse,split3_test_rmse,split4_test_rmse,mean_test_rmse,std_test_rmse,rank_test_rmse,split0_test_mae,split1_test_mae,...,rank_test_mae,mean_fit_time,std_fit_time,mean_test_time,std_test_time,params,param_n_epochs,param_lr_all,param_reg_all,param_n_factors
0,0.771657,0.783685,0.732995,0.841491,0.732773,0.772520,0.040061,116,0.629705,0.623394,...,120,0.002051,0.000591,0.002262,0.000346,"{'n_epochs': 5, 'lr_all': 0.002, 'reg_all': 0....",5,0.002,0.4,5
1,0.771949,0.783330,0.732887,0.841079,0.731752,0.772199,0.040124,113,0.630108,0.622918,...,116,0.001474,0.000099,0.001727,0.000208,"{'n_epochs': 5, 'lr_all': 0.002, 'reg_all': 0....",5,0.002,0.4,10
2,0.771623,0.784563,0.733235,0.840870,0.732986,0.772655,0.039809,120,0.628784,0.623415,...,117,0.001589,0.000063,0.001519,0.000073,"{'n_epochs': 5, 'lr_all': 0.002, 'reg_all': 0....",5,0.002,0.4,15
3,0.770473,0.784390,0.733063,0.841560,0.730773,0.772052,0.040522,107,0.628525,0.623558,...,112,0.001553,0.000089,0.001306,0.000078,"{'n_epochs': 5, 'lr_all': 0.002, 'reg_all': 0....",5,0.002,0.4,20
4,0.771778,0.784096,0.731556,0.838529,0.732144,0.771621,0.039487,102,0.630172,0.624010,...,110,0.001942,0.000032,0.001204,0.000010,"{'n_epochs': 5, 'lr_all': 0.002, 'reg_all': 0....",5,0.002,0.4,30


In [45]:
model.fit(trainset)

In [46]:
prediction = model.test(testset)

In [47]:
accuracy.rmse(prediction)

RMSE: 0.7398


0.739845653123489